In [1]:
from flask import Flask, jsonify
from flask_restful import Api, Resource, reqparse
from flask_sqlalchemy import SQLAlchemy
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity
from werkzeug.security import generate_password_hash, check_password_hash

# Initialize Flask app, API, and JWT
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///db.sqlite3'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['JWT_SECRET_KEY'] = 'super-secret-key'  # Change this in production

api = Api(app)
db = SQLAlchemy(app)
jwt = JWTManager(app)

# Define database models
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    password_hash = db.Column(db.String(120), nullable=False)
    role = db.Column(db.String(20), nullable=False, default='user')

    def set_password(self, password):
        self.password_hash = generate_password_hash(password)

    def check_password(self, password):
        return check_password_hash(self.password_hash, password)

class Item(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(80), unique=True, nullable=False)
    price = db.Column(db.Float, nullable=False)
    category_id = db.Column(db.Integer, db.ForeignKey('category.id'), nullable=False)

class Category(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(80), unique=True, nullable=False)
    items = db.relationship('Item', backref='category', lazy=True)

# Create database tables
@app.before_first_request
def create_tables():
    db.create_all()

# Request parsers
user_parser = reqparse.RequestParser()
user_parser.add_argument('username', type=str, required=True, help='This field cannot be blank')
user_parser.add_argument('password', type=str, required=True, help='This field cannot be blank')
user_parser.add_argument('role', type=str, required=False, choices=('user', 'admin'), default='user')

item_parser = reqparse.RequestParser()
item_parser.add_argument('name', type=str, required=True, help='This field cannot be blank')
item_parser.add_argument('price', type=float, required=True, help='This field cannot be blank')
item_parser.add_argument('category_id', type=int, required=True, help='This field cannot be blank')

category_parser = reqparse.RequestParser()
category_parser.add_argument('name', type=str, required=True, help='This field cannot be blank')

# Resource classes
class UserRegister(Resource):
    def post(self):
        data = user_parser.parse_args()
        if User.query.filter_by(username=data['username']).first():
            return {'message': 'User already exists'}, 400

        new_user = User(username=data['username'], role=data['role'])
        new_user.set_password(data['password'])
        db.session.add(new_user)
        db.session.commit()

        return {'message': 'User created successfully'}, 201

class UserLogin(Resource):
    def post(self):
        data = user_parser.parse_args()
        user = User.query.filter_by(username=data['username']).first()
        if user and user.check_password(data['password']):
            access_token = create_access_token(identity={'id': user.id, 'role': user.role})
            return {'access_token': access_token}, 200
        return {'message': 'Invalid credentials'}, 401

class ItemResource(Resource):
    @jwt_required()
    def get(self, item_id):
        item = Item.query.get_or_404(item_id)
        return {'name': item.name, 'price': item.price, 'category': item.category.name}, 200

    @jwt_required()
    def put(self, item_id):
        data = item_parser.parse_args()
        item = Item.query.get_or_404(item_id)
        identity = get_jwt_identity()
        if identity['role'] != 'admin':
            return {'message': 'Admin privilege required'}, 403

        item.name = data['name']
        item.price = data['price']
        item.category_id = data['category_id']
        db.session.commit()
        return {'message': 'Item updated'}, 200

    @jwt_required()
    def delete(self, item_id):
        item = Item.query.get_or_404(item_id)
        identity = get_jwt_identity()
        if identity['role'] != 'admin':
            return {'message': 'Admin privilege required'}, 403

        db.session.delete(item)
        db.session.commit()
        return {'message': 'Item deleted'}, 200

class ItemList(Resource):
    @jwt_required()
    def get(self):
        items = Item.query.all()
        return [{'id': item.id, 'name': item.name, 'price': item.price, 'category': item.category.name} for item in items], 200

    @jwt_required()
    def post(self):
        data = item_parser.parse_args()
        if Item.query.filter_by(name=data['name']).first():
            return {'message': 'Item already exists'}, 400

        identity = get_jwt_identity()
        if identity['role'] != 'admin':
            return {'message': 'Admin privilege required'}, 403

        new_item = Item(name=data['name'], price=data['price'], category_id=data['category_id'])
        db.session.add(new_item)
        db.session.commit()
        return {'message': 'Item created successfully'}, 201

class CategoryResource(Resource):
    @jwt_required()
    def get(self, category_id):
        category = Category.query.get_or_404(category_id)
        return {'id': category.id, 'name': category.name}, 200

    @jwt_required()
    def put(self, category_id):
        data = category_parser.parse_args()
        category = Category.query.get_or_404(category_id)
        identity = get_jwt_identity()
        if identity['role'] != 'admin':
            return {'message': 'Admin privilege required'}, 403

        category.name = data['name']
        db.session.commit()
        return {'message': 'Category updated'}, 200

    @jwt_required()
    def delete(self, category_id):
        category = Category.query.get_or_404(category_id)
        identity = get_jwt_identity()
        if identity['role'] != 'admin':
            return {'message': 'Admin privilege required'}, 403

        db.session.delete(category)
        db.session.commit()
        return {'message': 'Category deleted'}, 200

class CategoryList(Resource):
    @jwt_required()
    def get(self):
        categories = Category.query.all()
        return [{'id': category.id, 'name': category.name} for category in categories], 200

    @jwt_required()
    def post(self):
        data = category_parser.parse_args()
        if Category.query.filter_by(name=data['name']).first():
            return {'message': 'Category already exists'}, 400

        identity = get_jwt_identity()
        if identity['role'] != 'admin':
            return {'message': 'Admin privilege required'}, 403

        new_category = Category(name=data['name'])
        db.session.add(new_category)
        db.session.commit()
        return {'message': 'Category created successfully'}, 201

# Register resource endpoints
api.add_resource(UserRegister, '/register')
api.add_resource(UserLogin, '/login')
api.add_resource(ItemResource, '/item/<int:item_id>')
api.add_resource(ItemList, '/items')
api.add_resource(CategoryResource, '/category/<int:category_id>')
api.add_resource(CategoryList, '/categories')

# Run the application
if __name__ == '__main__':
    app.run(debug=True)


ModuleNotFoundError: No module named 'flask_restful'